# Algoritma Apriori Dataset Labor+Relations

# Marcello Rasel Hidayatullah <br>
# 1301184019

In [2]:
pip install apyori

  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5974 sha256=19a96d9770537a8d56dad40ef7a203774ebda5ddcebccaca34361d5f913ecf17
  Stored in directory: /root/.cache/pip/wheels/cb/f6/e1/57973c631d27efd1a2f375bd6a83b2a616c4021f24aab84080
Successfully built apyori


In [174]:
import numpy as np 
import pandas as pd 
import plotly.express as px
import apyori
from apyori import apriori

#Import Datasets

In [175]:
df = pd.read_csv('labor-negotiations.csv')
df

,duration,wage-increase-first-year,wage-increase-second-year,wage-increase-third-year,cost-of-living-adjustment,working-hours,pension,standby-pay,shift-differential,education-allowance,statutory-holidays,vacation,longterm-disability-assistance,contribution-to-dental-plan,bereavement-assistance,contribution-to-health-plan,class
0,1,5,?,?,?,40,?,?,2,?,11,average,?,?,yes,?,good
1,2,4.5,5.8,?,?,35,ret_allw,?,?,yes,11,below_average,?,full,?,full,good
2,?,?,?,?,?,38,empl_contr,?,5,?,11,generous,yes,half,yes,half,good
3,3,3.7,4,5,tc,?,?,?,?,yes,?,?,?,?,yes,?,good
4,3,4.5,4.5,5,?,40,?,?,?,?,12,average,?,half,yes,half,good
5,2,2,2.5,?,?,35,?,?,6,yes,12,average,?,?,?,?,good
6,3,4,5,5,tc,?,empl_contr,?,?,?,12,generous,yes,none,yes,half,good
7,3,6.9,4.8,2.3,?,40,?,?,3,?,12,below_average,?,?,?,?,good
8,2,3,7,?,?,38,?,12,25,yes,11,below_average,yes,half,yes,?,good
9,1,5.7,?,?,none,40,empl_contr,?,4,?,11,generous,yes,full,?,?,good


In [176]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   duration                        57 non-null     object
 1   wage-increase-first-year        57 non-null     object
 2   wage-increase-second-year       57 non-null     object
 3   wage-increase-third-year        57 non-null     object
 4   cost-of-living-adjustment       57 non-null     object
 5   working-hours                   57 non-null     object
 6   pension                         57 non-null     object
 7   standby-pay                     57 non-null     object
 8   shift-differential              57 non-null     object
 9   education-allowance             57 non-null     object
 10  statutory-holidays              57 non-null     object
 11  vacation                        57 non-null     object
 12  longterm-disability-assistance  57 non-null     obje

# Pre Processing

## Check and Handling Missing Values

In [177]:
# Mengganti nilai '?' dengan 'Nan'
df.replace('?', np.nan, inplace= True)
df.head()

,duration,wage-increase-first-year,wage-increase-second-year,wage-increase-third-year,cost-of-living-adjustment,working-hours,pension,standby-pay,shift-differential,education-allowance,statutory-holidays,vacation,longterm-disability-assistance,contribution-to-dental-plan,bereavement-assistance,contribution-to-health-plan,class
0,1,5,NaN,NaN,NaN,40,NaN,NaN,2,NaN,11,average,NaN,NaN,yes,NaN,good
1,2,4.5,5.8,NaN,NaN,35,ret_allw,NaN,NaN,yes,11,below_average,NaN,full,NaN,full,good
2,NaN,NaN,NaN,NaN,NaN,38,empl_contr,NaN,5,NaN,11,generous,yes,half,yes,half,good
3,3,3.7,4,5,tc,NaN,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,yes,NaN,good
4,3,4.5,4.5,5,NaN,40,NaN,NaN,NaN,NaN,12,average,NaN,half,yes,half,good


In [178]:
df.shape

(57, 17)

In [179]:
df.isnull().sum()

duration                           1
wage-increase-first-year           1
wage-increase-second-year         11
wage-increase-third-year          42
cost-of-living-adjustment         20
working-hours                      6
pension                           30
standby-pay                       48
shift-differential                26
education-allowance               35
statutory-holidays                 4
vacation                           6
longterm-disability-assistance    29
contribution-to-dental-plan       20
bereavement-assistance            27
contribution-to-health-plan       20
class                              0
dtype: int64

In [180]:
# Mengisi missing values dengan modus
for column in df:
    modus = df[column].mode()
    df[column].fillna(modus[0], inplace=True)

In [181]:
df.isnull().sum()

duration                          0
wage-increase-first-year          0
wage-increase-second-year         0
wage-increase-third-year          0
cost-of-living-adjustment         0
working-hours                     0
pension                           0
standby-pay                       0
shift-differential                0
education-allowance               0
statutory-holidays                0
vacation                          0
longterm-disability-assistance    0
contribution-to-dental-plan       0
bereavement-assistance            0
contribution-to-health-plan       0
class                             0
dtype: int64

In [182]:
df.head()

,duration,wage-increase-first-year,wage-increase-second-year,wage-increase-third-year,cost-of-living-adjustment,working-hours,pension,standby-pay,shift-differential,education-allowance,statutory-holidays,vacation,longterm-disability-assistance,contribution-to-dental-plan,bereavement-assistance,contribution-to-health-plan,class
0,1,5,4,5,none,40,empl_contr,2,2,no,11,average,yes,half,yes,full,good
1,2,4.5,5.8,5,none,35,ret_allw,2,3,yes,11,below_average,yes,full,yes,full,good
2,2,2,4,5,none,38,empl_contr,2,5,no,11,generous,yes,half,yes,half,good
3,3,3.7,4,5,tc,40,empl_contr,2,3,yes,11,below_average,yes,half,yes,full,good
4,3,4.5,4.5,5,none,40,empl_contr,2,3,no,12,average,yes,half,yes,half,good


Mengubah nama object agar mempermudah memahami Association Rule

In [183]:
df['duration']=df['duration'].replace(['1','2','3'], ['duration1','duration2','duration3'])
df['wage-increase-first-year']=df['wage-increase-first-year'].replace(['2', '2.1', '2.5', '2.8', '3', '3.5', '3.7', '4', '4.3', '4.5',
       '4.6', '5', '5.7', '6', '6.4', '6.9', '7'], ['wage-increase-first-year2', 'wage-increase-first-year2.1', 'wage-increase-first-year2.5', 'wage-increase-first-year2.8', 'wage-increase-first-year3', 'wage-increase-first-year3.5', 
        'wage-increase-first-year3.7', 'wage-increase-first-year4', 'wage-increase-first-year4.3', 'wage-increase-first-year4.5', 'wage-increase-first-year4.6', 'wage-increase-first-year5', 'wage-increase-first-year5.7', 'wage-increase-first-year6', 
        'wage-increase-first-year6.4', 'wage-increase-first-year6.9', 'wage-increase-first-year7'])
df['wage-increase-second-year']=df['wage-increase-second-year'].replace(['2', '2.5', '3', '3.5', '4', '4.4', '4.5', '4.6', '4.8', '5', '5.3', '5.8', '6', '6.4', '7'], 
       ['wage-increase-second-year2', 'wage-increase-second-year2.5', 'wage-increase-second-year3', 'wage-increase-second-year3.5', 'wage-increase-second-year4', 'wage-increase-second-year4.4', 'wage-increase-second-year4.5', 
        'wageSecond4.6', 'wageSecond4.8', 'wageSecond5',
       'wage-increase-second-year5.3', 'wage-increase-second-year5.8', 'wage-increase-second-year6', 'wage-increase-second-year6.4', 'wage-increase-second-year7'])
df['wage-increase-third-year']=df['wage-increase-third-year'].replace(['2', '2.1', '2.3', '2.5', '4', '4.5', '4.6', '5', '5.1'], 
      ['wage-increase-third-year2', 'wage-increase-third-year2.1', 'wage-increase-third-year2.3', 'wage-increase-third-year2.5', 'wage-increase-third-year4', 'wage-increase-third-year4.5', 'wage-increase-third-year4.6', 'wage-increase-third-year5', 'wage-increase-third-year5.1'])
df['working-hours']=df['working-hours'].replace(['27', '33', '35', '36', '37', '38', '39', '40'], ['working-hours27', 'working-hours33', 'working-hours35', 'working-hours36', 'working-hours37', 'working-hours38', 'working-hours39', 'working-hours40'])
df['standby-pay']=df['standby-pay'].replace(['10', '12', '13', '14', '2', '4', '8'], ['standby-pay10', 'standby-pay12', 'standby-pay13', 'standby-pay14', 'standby-pay2', 'standby-pay4', 'standby-pay8'])
df['shift-differential']=df['shift-differential'].replace(['0', '1', '10', '14', '2', '25', '3', '4', '5', '6'], ['shift0', 'shift1', 'shift10', 'shift14', 'shift2', 'shift25', 'shift3', 'shift4', 'shift5', 'shift6'])
df['statutory-holidays']=df['statutory-holidays'].replace(['10', '11', '12', '13', '15', '9'], ['statutory-holidays10', 'statutory-holidays11', 'statutory-holidays12', 'statutory-holidays13', 'statutory-holidays15', 'statutory-holidays9'])
df.head()

,duration,wage-increase-first-year,wage-increase-second-year,wage-increase-third-year,cost-of-living-adjustment,working-hours,pension,standby-pay,shift-differential,education-allowance,statutory-holidays,vacation,longterm-disability-assistance,contribution-to-dental-plan,bereavement-assistance,contribution-to-health-plan,class
0,duration1,wage-increase-first-year5,wage-increase-second-year4,wage-increase-third-year5,none,working-hours40,empl_contr,standby-pay2,shift2,no,statutory-holidays11,average,yes,half,yes,full,good
1,duration2,wage-increase-first-year4.5,wage-increase-second-year5.8,wage-increase-third-year5,none,working-hours35,ret_allw,standby-pay2,shift3,yes,statutory-holidays11,below_average,yes,full,yes,full,good
2,duration2,wage-increase-first-year2,wage-increase-second-year4,wage-increase-third-year5,none,working-hours38,empl_contr,standby-pay2,shift5,no,statutory-holidays11,generous,yes,half,yes,half,good
3,duration3,wage-increase-first-year3.7,wage-increase-second-year4,wage-increase-third-year5,tc,working-hours40,empl_contr,standby-pay2,shift3,yes,statutory-holidays11,below_average,yes,half,yes,full,good
4,duration3,wage-increase-first-year4.5,wage-increase-second-year4.5,wage-increase-third-year5,none,working-hours40,empl_contr,standby-pay2,shift3,no,statutory-holidays12,average,yes,half,yes,half,good


# Algoritma Apriori

In [184]:
relation = []
for i in range(0, 57):
    relation.append([str(df.values[i,j]) for j in range(0, 16)])
rules = apriori(relation, min_support = 0.02, min_confidence = 0.7, min_lift = 1.1, max_length = 2, target = "rules")
association_results = list(rules)

# Rule Association

In [185]:
for item in association_results:
    
    count = item[0]
    items = [x for x in count]
    
    print("RULE")
    print("Antecedents Consequents : ", items[0], " -> " + items[1])
    print("Support : ", str(item[1]))
    print("Confidence : ",str(item[2][0][2]))
    print("Lift : ", str(item[2][0][3]))
    
    print("===================================================================================================")

RULE
Antecedents Consequents :  average  -> shift3
Support :  0.21052631578947367
Confidence :  0.7058823529411765
Lift :  1.2192513368983957
RULE
Antecedents Consequents :  average  -> standby-pay2
Support :  0.2982456140350877
Confidence :  1.0
Lift :  1.1176470588235294
RULE
Antecedents Consequents :  average  -> wage-increase-second-year2.5
Support :  0.08771929824561403
Confidence :  0.8333333333333334
Lift :  2.794117647058824
RULE
Antecedents Consequents :  average  -> wage-increase-third-year5
Support :  0.2807017543859649
Confidence :  0.9411764705882353
Lift :  1.1414267834793492
RULE
Antecedents Consequents :  half  -> below_average
Support :  0.3157894736842105
Confidence :  0.75
Lift :  1.125
RULE
Antecedents Consequents :  shift2  -> below_average
Support :  0.05263157894736842
Confidence :  0.75
Lift :  1.78125
RULE
Antecedents Consequents :  wage-increase-first-year2.8  -> below_average
Support :  0.03508771929824561
Confidence :  1.0
Lift :  2.375
RULE
Antecedents Cons

In [186]:
hasil = pd.DataFrame(association_results)
hasil

,items,support,ordered_statistics
0,"(average, shift3)",0.210526,"[((average), (shift3), 0.7058823529411765, 1.2..."
1,"(average, standby-pay2)",0.298246,"[((average), (standby-pay2), 1.0, 1.1176470588..."
2,"(average, wage-increase-second-year2.5)",0.087719,"[((wage-increase-second-year2.5), (average), 0..."
3,"(average, wage-increase-third-year5)",0.280702,"[((average), (wage-increase-third-year5), 0.94..."
4,"(half, below_average)",0.315789,"[((below_average), (half), 0.75, 1.125)]"
...,...,...,...
189,"(wage-increase-second-year4.5, wage-increase-t...",0.087719,"[((wage-increase-second-year4.5), (wage-increa..."
190,"(wage-increase-second-year4.5, working-hours40)",0.070175,"[((wage-increase-second-year4.5), (working-hou..."
191,"(wageSecond5, wage-increase-third-year5)",0.070175,"[((wageSecond5), (wage-increase-third-year5), ..."
192,"(working-hours38, wage-increase-third-year5)",0.210526,"[((working-hours38), (wage-increase-third-year..."


In [187]:
hasil_urut = pd.DataFrame(columns=('Items','Antecedent','Consequent','Support','Confidence','Lift'))

Support =[]
Confidence = []
Lift = []
Items = []
Antecedent = []
Consequent=[]

for RelationRecord in association_results:
    for ordered_stat in RelationRecord.ordered_statistics:
        Support.append(RelationRecord.support)
        Items.append(RelationRecord.items)
        Antecedent.append(ordered_stat.items_base)
        Consequent.append(ordered_stat.items_add)
        Confidence.append(ordered_stat.confidence)
        Lift.append(ordered_stat.lift)

hasil_urut['Items'] = list(map(set, Items))                                   
hasil_urut['Antecedent'] = list(map(set, Antecedent))
hasil_urut['Consequent'] = list(map(set, Consequent))
hasil_urut['Support'] = Support
hasil_urut['Confidence'] = Confidence
hasil_urut['Lift']= Lift

In [188]:
hasil_urut.sort_values(by ='Lift', ascending = False, inplace = True)
hasil_urut.head()

,Items,Antecedent,Consequent,Support,Confidence,Lift
166,"{wage-increase-first-year6, statutory-holidays9}",{wage-increase-first-year6},{statutory-holidays9},0.035088,1.00,14.250
174,"{wage-increase-first-year3.5, wage-increase-th...",{wage-increase-third-year4.6},{wage-increase-first-year3.5},0.035088,1.00,11.400
163,"{wage-increase-first-year3.5, statutory-holida...",{statutory-holidays13},{wage-increase-first-year3.5},0.052632,1.00,11.400
176,"{wageSecond5, wage-increase-first-year4}",{wageSecond5},{wage-increase-first-year4},0.052632,0.75,7.125
10,"{wage-increase-first-year2.8, duration1}",{wage-increase-first-year2.8},{duration1},0.035088,1.00,5.700
